In [1]:
# Install dependencies
%pip install requests html2text langchain openai tiktoken

     ---------------------------------------- 0.0/56.5 kB ? eta -:--:--
     ------------- ------------------------ 20.5/56.5 kB 640.0 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/56.5 kB 640.0 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/56.5 kB 640.0 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/56.5 kB 640.0 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/56.5 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 56.5/56.5 kB 246.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     - -------------------------------------- 0.0/1.0 MB 1.4 MB/s eta 0:00:01
     - -------------------------------------- 0.0/1.0 MB 991.0 kB/s eta 0:00:01
     - -------------------------------------- 0.0/1.0 MB 991.0 kB/s eta 0:00:01
     - ------------------------------

  DEPRECATION: html2text is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
# Get environment variables
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env file

secret = os.getenv('SMARTLEAD_API_KEY')
openai_secret = os.getenv('OPENAI_KEY')
apollo_secret = os.getenv('APOLLO_API_KEY')


In [17]:
print(openai_secret)

None


In [2]:
# Import dependencies
import requests
import csv
import json
import html2text
import ast
from openai import OpenAI as OpenAIPython
from langchain.llms import OpenAI
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryBufferMemory
from langchain.agents import initialize_agent, AgentType
from langchain_openai  import ChatOpenAI
OpenAI_LLM = OpenAI(temperature=0.6,api_key=openai_secret)
ChatOpenAI_LLM = ChatOpenAI(temperature=0, model="gpt-4",api_key=openai_secret)
client = OpenAIPython(
    # This is the default and can be omitted
    api_key=openai_secret,
)
from datetime import datetime
current_datetime = datetime.utcnow()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_10868\3293446694.py:14: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  OpenAI_LLM = OpenAI(temperature=0.6,api_key=openai_secret)


In [3]:
# Get all the campaigns
response = requests.get(f"https://server.smartlead.ai/api/v1/campaigns?api_key={secret}")
response = response.json()
print(response)

[{'id': 1485452, 'user_id': 86384, 'created_at': '2025-02-12T03:12:25.946Z', 'updated_at': '2025-02-12T17:00:21.137Z', 'status': 'DRAFTED', 'name': 'first campaign', 'track_settings': [], 'scheduler_cron_value': {'tz': 'Asia/Saigon', 'days': [1, 2, 3, 4, 5], 'endHour': '18:00', 'startHour': '09:00'}, 'min_time_btwn_emails': 20, 'max_leads_per_day': 100, 'stop_lead_settings': 'REPLY_TO_AN_EMAIL', 'enable_ai_esp_matching': False, 'send_as_plain_text': False, 'follow_up_percentage': 0, 'unsubscribe_text': '', 'parent_campaign_id': None, 'client_id': None}, {'id': 1485611, 'user_id': 86384, 'created_at': '2025-02-12T03:49:41.905Z', 'updated_at': '2025-02-12T17:00:21.137Z', 'status': 'COMPLETED', 'name': 'test campaign', 'track_settings': [], 'scheduler_cron_value': {'tz': 'Asia/Saigon', 'days': [1, 2, 3, 4, 5], 'endHour': '18:00', 'startHour': '09:00'}, 'min_time_btwn_emails': 5, 'max_leads_per_day': 100, 'stop_lead_settings': 'REPLY_TO_AN_EMAIL', 'enable_ai_esp_matching': False, 'send_as_

In [ ]:
with open('Lead_Data.csv',encoding='utf-8', mode='r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    # Skip the header row
    next(csv_reader)
    with open('final.csv', mode='w',encoding='utf-8', newline='') as final_csv:
      fieldnames = ['Message', 'Reply']
      writer = csv.DictWriter(final_csv, fieldnames=fieldnames)
      writer.writeheader()
      # Iterate over each row in the CSV file
      for row in csv_reader:
		      # Get the lead id from smartlead API
          lead_info = requests.get(f"https://server.smartlead.ai/api/v1/leads/?api_key={secret}&email={row[2]}")
          lead_info = lead_info.json()
          # Get message history from smartlead API
          message_history = requests.get(f"https://server.smartlead.ai/api/v1/campaigns/{1485611}/leads/{lead_info['id']}/message-history?api_key={secret}")
          message_history = message_history.json()
          # traverse message history array
          for index, message in enumerate(message_history['history']):
            if(message['type'] == "REPLY" and index != len(message_history['history'])):
	            # The conversation you will get from smartlead will be in html so we need to convert it into plain text
              plain_text = html2text.html2text(message['email_body'])
							# Prompt for LLM to format the email thread 
              prompt = f"""
                  Email Thread:
                  ---
                  {plain_text}
                  ---
                  You have given a email thread as a plain text and you have to return the latest email from it

                  You have to follow these steps to do it:
                  Step-1: Get the text which don't starts with '>' because every other text which starts with '>' is a old message in thread
                  (It will be at the starting of text and older messages will be at below this latest message)

                  The email thread format typically look like this:
                  ---
                  Hey john,
                  I am interested
                  Thanks,
                  shivam
                  > sentence 1 ....
                  > sentence 2 ....
                  > ...
                  >> sentence from older emails ...
                  >> sentence from older emails ....
                  >> ....
                  ---
                  For above example, the email message content will be:
                  Hey john,
                  I am interested
                  Thanks,
                  Shivam

                  Note: Please note that the above message content was just an example and your respond must be related to given plain text

                  Step-2: Once you got the latest email then get the message content from it and remove any extra blank space or unnecessary content from email
                  After these steps, Return the email message content in response

                  RESPONSE (Don't return anything except email message):
                """
              email_content = client.chat.completions.create(
                  model="gpt-4",
                  messages=[
                      {"role": "user", "content": prompt}
                  ]
              )
              email_content = email_content.choices[0].message.content
              reply = html2text.html2text(message_history["history"][index]["email_body"])
              print(reply)
              # Add the formatted data into new csv file
              writer.writerow({'Message': email_content, 'Reply': reply})

Processing message 2/2, type: REPLY, body:Hi Jason,

Sounds good, It would be interesting if you can tell me in details

  

Best,

Nhat  

Sent from my iPhone

  

> On 12 Feb 2025, at 14:56, info mation <info@jesselton.capital> wrote:  
>  
>

> ﻿ Test email for drip marketing campaign
>
> Hi Nhat 1,
>
> I’m Jason, Executive at Hyred, and I’m reaching out to see if you’re looking
> to build a top-tier team in Southeast Asia or Hong Kong. Recruitment here
> comes with unique challenges—but we make it seamless.
>
>  
>
>
> For 18 years, we’ve been helping companies like yours not just fill roles,
> but find the perfect fit—people who align with your vision, culture, and
> goals.
>
>  
>
>
> Here’s how we make hiring easier for you:
>
> ✔️ Local expertise – We know these markets inside out.
>
> ✔️ Tailored approach – We adapt to your hiring needs.
>
> ✔️ Culture-first mindset – Skills matter, but fit is everything.
>
>  
>
>
> Would you be open to a quick chat to share with us your hiri

In [79]:
from langchain.pydantic_v1 import BaseModel, Field
from typing import Type, List
from crewai.tools.structured_tool import CrewStructuredTool
# Input class for tool so that it can follow strict input parameter schema
class CategorizeEmailInput(BaseModel):
    conversation: str = Field(description="Email conversation array")

class CategorizeEmailTool(BaseTool):
		# Provide proper name and description for your tool
    name: str = "email_categorizer_tool"
    description: str = "use this tool when have email conversation history and you want to categorize this email"
    args_schema: Type[BaseModel] = CategorizeEmailInput

    def _run(self, conversation: str):
      prompt = f"""
        Email Conversation History:
        ---
        {conversation}
        ---
        You have given an array of conversation between Rohan Sawant and a client
        Your goal is to categorize this email based on the conversation history from the given categories:

        1. Meeting_Ready_Lead: they have shown positive intent and are interested in getting on a call
        2. Power: If they’re interested and we want to push for a call
        3. Question: If they have any question regarding anything
        4. Unsubscribe: They want to unsubscribe themselves from our email list
        5. OOO: They are out of office
        6. No_Longer_Works: They no longer works in the company
        7. Not_Interested: They are not interested
        8. Info: these are emails that don't fit into any of the above categories.

        Note: Your final response MUST BE the category name ONLY

        RESPONSE:
      """
      message = client.chat.completions.create(
          model="gpt-4",
          messages=[
              {"role": "user", "content": prompt}
          ]
      )
      category = message.choices[0].message.content
      return category

    def _arun(self, url: str):
        raise NotImplementedError(
            "categorise_email does not support async")

In [7]:
# Run the agent for your leads
campaign_id = 1485611
with open('Campaign_Leads.csv',encoding='utf-8', mode='r') as file:
  # Create a CSV reader object
  csv_reader = csv.reader(file)
  for index,row in enumerate(csv_reader):
    if index == 0:
      continue
    # Get lead ID
    lead_info = requests.get(f"https://server.smartlead.ai/api/v1/leads/?api_key={secret}&email={row[2]}")
    lead_info = lead_info.json()
    # print(lead_info)
    # Get conversation history of lead
    message_history = requests.get(f"https://server.smartlead.ai/api/v1/campaigns/{campaign_id}/leads/{lead_info['id']}/message-history?api_key={secret}")
    message_history = message_history.json()
    message_history = message_history["history"]
    # print(message_history)
    conversation_history = []
    # Format every message in conversation history
    for message in message_history:
      plain_text = html2text.html2text(message["email_body"])
      prompt = f"""
        Email Thread:
        ---
        {plain_text}
        ---
        You have given a email thread as a plain text and you have to return the latest email from it

        You have to follow these steps to do it:
        Step-1: Get the text which don't starts with '>' because every other text which starts with '>' is a old message in thread
        (It will be at the starting of text and older messages will be at below this latest message)

        The email thread format typically look like this:
        ---
        Hey john,
        I am interested
        Thanks,
        shivam
        > sentence 1 ....
        > sentence 2 ....
        > ...
        >> sentence from older emails ...
        >> sentence from older emails ....
        >> ....
        ---
        For above example, the email message content will be:
        Hey john,
        I am interested
        Thanks,
        Shivam

        Note: Please note that the above message content was just an example and your respond must be related to given plain text

        Step-2: Once you got the latest email then get the message content from it and remove any extra blank space or unnecessary content from email
        After these steps, Return the email message content in response

        RESPONSE (Don't return anything except email message):
      """
      email_content = client.chat.completions.create(
          model="gpt-4",
          messages=[
              {"role": "user", "content": prompt}
          ]
      )
      email_content = email_content.choices[0].message.content
      convo = {
          "sender": "info mation" if message["type"] == "SENT" else row[0],
          "message": email_content
      }
      conversation_history.append(convo)
    # Prompt for our agent
    prompt = f"""
      Email conversation history:
      ---
      {conversation_history}
      ---
      Lead Name: {row[0]}
      Lead Email: {row[2]}

      Sender of last message: {conversation_history[len(conversation_history) - 1]["sender"]}
    """

    print(prompt)


      Email conversation history:
      ---
      [{'sender': 'info mation', 'message': 'Hi Trang,\n\nI’m Jason, Executive at Hyred, and I’m reaching out to see if you’re looking\nto build a top-tier team in Southeast Asia or Hong Kong. Recruitment here\ncomes with unique challenges—but we make it seamless.\n\nFor 18 years, we’ve been helping companies like yours not just fill roles, but\nfind the perfect fit—people who align with your vision, culture, and goals.\n\nHere’s how we make hiring easier for you:\n\n✔️ Local expertise – We know these markets inside out.\n✔️ Tailored approach – We adapt to your hiring needs.\n✔️ Culture-first mindset – Skills matter, but fit is everything.\n\nWould you be open to a quick chat to share with us your hiring challenges and\nexplore how we can help you build a talented team?\n\nLooking forward to connecting!\n\nBest,\n\nJason\n\nHyred Recruitment Team\n\n+84 415915232'}]
      ---
      Lead Name: Trang
      Lead Email: trang.pham@jesselton.capi